## Web Service Client

En este notebook vamos a consumir un servicio de machine learning. El servicio debe ser levantando con el script server.py por separado.

In [1]:
import requests

# url del servicio
url = 'http://127.0.0.1:1080/predict'

# cuerpo del mensaje
body = {
    "petal_length": "2.1",
    "sepal_length": "2",
    "petal_width": "0.5",
    "sepal_width": "3"
}

# enviamos los datos por POST
response = requests.post(url, data=body)

# imprimimos el mensaje
print(response.json())

{'Prediccion': 2}


### EJERCICIO 1: 

Modifique el server.py para que acepte requests por GET. Modifique el codigo del cliente para enviar los datos por GET

In [2]:
# DEFINA SU CLIENTE GET AQUI:
response = requests.get(url, data=body)
print(response.json())

{'Prediccion': 2}


### EJERCICIO 2:

* Entrene un modelo de machine learning con MNIST y salve su modelo en un archivo pickle. [https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html]
* Modifique el server.py para que agregue la ruta /predict-number y que sea capaz de predecir si una imagen pertecene a un numero del 0 al 9
* Va a enviar la imagen desde el cliente (este notebook) como un base64 hacia el server.py
* el server.py va recibir la imagen la reconstruye en una imagen nuevamente y la manda al modelo de ML
* la imagen que ingresa al server.py debe salvarla en un folder (images)

In [3]:
## Entrenar MNIST y salvarlo en pickle
from sklearn.datasets import fetch_openml, load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, Int64TensorType, StringTensorType, DoubleTensorType, SequenceType, DictionaryType
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pickle
import requests
import base64
import cv2

In [4]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

In [5]:
MnistModel = Pipeline(steps=[('scaler',MinMaxScaler()),
                             ('pca',PCA(n_components=150)),
                             ('lr',LogisticRegression(C=5))])
MnistModel.fit(X, y)

Pipeline(steps=[('scaler', MinMaxScaler()), ('pca', PCA(n_components=150)),
                ('lr', LogisticRegression(C=5))])

In [6]:
pkl_filename = "MnistLR.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(MnistModel, file)

In [7]:
initial_types = [('input', FloatTensorType([1, X.shape[1]]))]
onx = convert_sklearn(MnistModel, initial_types=initial_types)
with open("MnistLR.onnx", "wb") as file:
    file.write(onx.SerializeToString())

In [43]:
img_path = "NumberToProcess.png"
img = cv2.bitwise_not(cv2.imread(img_path, 0))
encoded_string = base64.b64encode(img)

In [44]:
url = 'http://127.0.0.1:1080/predict-number'
body = {"ImageBase64": encoded_string}
response = requests.get(url, data=body)
print(response.json())

{'Prediccion': 2}
